In [ ]:
import os

from astropy.time import Time, TimeDelta
import numpy as np
import pandas as pd

from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient("ncsa_teststand_efd")

In [ ]:
info = {}
info['table_name'] = 'lsst.sal.MTM1M3.hardpointActuatorData'
info['expected_rate'] = 50  # Hz
info['output_file'] = 'M1M3_EFD_HardpointData_Insertion_Tracking.csv'

In [ ]:
time_increment = TimeDelta(600, format='sec', scale='tai')
base_time = Time("2020-05-30T00:00:00", scale='tai')
now = Time("2020-06-01T12:00:00", scale='tai')

timestamp_arr = []
expected_messages_arr = []
received_messages_arr = []
percent_loss_arr = []
inc = 0
while base_time < now:
    #if inc % 5 == 0:
    print(base_time)
    
    df1 = await client.select_time_series(info['table_name'], '*', base_time, time_increment, is_window=True)
    
    expected_num_messages = round(time_increment.sec) * info['expected_rate']
    num_received_messages = df1.shape[0]
    delta_messages = expected_num_messages - num_received_messages
    percent_loss = (delta_messages / expected_num_messages) * 100
    
    timestamp_arr.append(base_time)
    expected_messages_arr.append(expected_num_messages)
    received_messages_arr.append(num_received_messages)
    percent_loss_arr.append(percent_loss)
    
    base_time += time_increment
    inc += 1
print("Next Time Start:", base_time)

In [ ]:
timestamps = np.array(timestamp_arr)
expected_messages = np.array(expected_messages_arr)
received_messages = np.array(received_messages_arr)
percent_loss = np.array(percent_loss_arr)

In [ ]:
output_df = pd.DataFrame({"Expected Messages": expected_messages, 
                          "Received Messages": received_messages, 
                          "Percent Loss": percent_loss}, index=timestamps)

In [ ]:
output_file = info['output_file']
header = not os.path.exists(output_file)
output_df.to_csv(output_file, mode='a+', index_label="Timestamps",
                 float_format='%0.4f', header=header)